# Exercise 8: hydrocarbons on platinum: functional-sensitive adsorption energies

Date: Wednesday, 19. April 2023

**Submission deadline**: Sunday, 23. April 2023 (or Sunday, 30. April 2023)

## 0. Introduction to the problem

In this exercise we will continue our study of density-functional theory (DFT) by considering the adsorption of three hydrocarbons -- benzene, methane, and ethane -- on a platinum slab. In principle, there are three candidates as adsorption sites, namely directly centered on top of a Pt atom, on the bridge connecting two Pt atoms, or on a hollow juncture in the middle of four Pt atoms.

![Three different hydrocarbons on the Pt(111) surface.](hydrocarbons_on_pt.png "Figure 1")

The aim of this exercise is to simulate the chemisorption of these hydrocarbons on platinum: in chemisorption, interactions between the molecule and substrate lead to bonds being formed, such that the bonds within the molecule and the surface are rearranged. We will be making use of [this paper [1]](https://pubs.rsc.org/en/content/articlepdf/2015/cp/c5cp04534g) in the course of this exercise. The first step is to make use of the definition of the adsorption energy $E_{ads}$:

\begin{equation*}
E_{ads} =  E_{total} - \sum_{slab, \, molecules} E_{separate \, system}
\end{equation*}

Thus, for our system:
\begin{equation}
E_{ads} = E_{total} - E_{Pt \, slab} - E_{molecule}
\end{equation}

Practically this means that three geometry optimizations are necessary: the first consisting of the total system of the molecule and the slab; the second with only the molecule; and the third with only the slab. However, not all DFT functionals calculations can reproduce this result. Because of their different properties and approximations for the exchange–correlation (XC) part, *different DFT functionals predict different properties*, especially in regards to van der Waals interactions, which we have indeed already met in the third exercise via the Lennard-Jones potential.

Every functional has its advantages and disadvantages and can be more or less suitable for the specific system and/or property that you want to study. Thus, it is necessary to understand the limits of the functional that you want to use, e.g. searching in the literature or performing some preliminary benchmark calculations. 

To give you a flavour of how properties can change, in this exercise we (you) will investigate one XC functional of the local-density approximation (LDA) type, as well as two from the same family of approximations to the XC energy, namely the generalized gradient approximation (GGA) functional, and we will use the GGA with and without vdW corrections: thus, in this exercise, we will examine a total of three different XC functionals. Different from exercise 3, this time, we will include the vdW correction via the BEEF-vdW XC functional instead of via a simple LJ potential.

As suggested by the name, the LDA is a first order approximations, and so depends solely upon the value of the electronic density at each point in space. On the other hand, GGA formalism goes a step further, accounting also for the local change of the electron density, with the inclusion of the electron density gradient in the functional. GGA functionals go through another division based on the origin of their parameters: They can be either empirical, i.e. derived from fitting of data coming from experiments and more sophisticated techniques, or determined from properties of the homogeneous electron gas as well as exact constraints. We will compare a GGA of this latter type, proposed by Perdew, Burke and Ernzerhof, and thus named PBE, to the LDA functional.

LDA generally overestimates the bond strength between the molecule and the surface, since it is more suitable to describe strong bonds. This leads to a calculated overbinding, such as short adsorption bond lengths, large chemisorption energies, and high molecule–surface vibrational frequencies. The error in the chemisorption energy can be so large that LDA may predict the wrong chemisorption site. Finally, by considering vdW corrections via the BEEF-vdW functional, we take into account non-local effects which may play a role in the chemisorption process.

## 1: Benzene $\text{C}_6\text{H}_6$ on Pt(111)

We begin by logging into our accounts on JupyterHub on Euler in a similar manner as described in the previous exercises, and `pull` the newest exercise files from the GitHub repository, making sure that you are in the directory for the course:
```bash
    $ cd ~/Molecular-and-Materials-Modelling-FS2023
    $ git init
  # $ git stash
    $ git pull https://github.com/ramador09/Molecular-and-Materials-Modelling-FS2023.git
```  

First, execute the following two cells to import necessary modules and libraries, and to define the `view_structure()` function, which we will make ample use of in the course of this exercise:

In [75]:
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase.build import fcc111,add_adsorbate,molecule
import matplotlib.pyplot as plt
import nglview as nv

In [76]:
def view_structure(system):
    t = nv.ASEStructure(system) 
    w = nv.NGLWidget(t, gui=True)
   # w.add_spacefill()
    return w

Next, we create an instance of the `molecule` class by starting with the benzene molecule. ASE is "smart" enough to know that we mean the benzene molecule when we simply create an instance of the `molecule` class and feed it the string `C6H6`:

In [77]:
benzene=molecule('C6H6')

In [78]:
view_structure(benzene)

NGLWidget()

Now, having initialised the `benzene` object, we can place it on the surface of Pt(111) by executing the following cell. **Note**: the lateral size of the supercell is related with the periodic simulation box parameters in the CP2K input file. These have already been chosen and adjusted to the (6,6,3) case, so please do not change the values in the `size` argument without adjusting the `CELL` parameters in the CP2K input file. If this sentence confuses you, just ignore it and change as little as possible ;-)

In [93]:
benz_pt_slab=fcc111('Pt',size=(6,6,3))
add_adsorbate(benz_pt_slab,benzene,height=2.25,position=(10,7))
benz_pt_slab.center(vacuum=10, axis=2)
view_structure(benz_pt_slab)

NGLWidget()

Use the `get_cell()` method on this `Atoms` object to view the unit cell of the system. This will serve as a good sanity check during the calculations, and moreover, we will need to use these values in the `CELL` subsection of the CP2K input file:

In [94]:
benz_pt_slab.get_cell()

Cell([[16.6311514935076, 0.0, 0.0], [8.3155757467538, 14.402999687565085, 0.0], [0.0, 0.0, 26.776426110446664]])

Save the slab-and-molecule composite system by executing the `write` function from the `ase.io` library:

In [95]:
write(filename='benzene_on_pt.xyz',images=benz_pt_slab)

## Connecting to the Eiger supercomputer

Due to the bugs and glitches on Euler, we will try to submit and run calculations on the Piz Eiger supercomputer in Lugano. We will do this by connecting via ssh to Eiger from Euler. In the current repository, you will see a directory called `eiger_connect`. Open the `README.txt` file and execute each of the lines, line by line, in your command line.

Once you `ssh` to Eiger, you will be in Daniele's `$SCRATCH` directory on Eiger. It is here where we will submit the calculations.

The `run` file is the submission script, and, for the purposes of this exercise, should **not** be modified. In the current `$SCRATCH` directory, make a new directory with your username on Euler:

```bash
$ mkdir <username>
$ cd <username>
```

It is within this `<username>` subdirectory that each student will submit calculations. Copy the `run` file from the parent `$SCRATCH` directory into the current `<username>` directory:

```bash
$ cp ../run .
```

Make subdirectories in your `<username>` directory as you see fit. In order to submit a job, make sure you're in the appropriate subdirectory containing your input and geometry files (and make sure the `run` script is there also of course), and issue

```bash
$ sbatch -J <username>  --export=ALL,root=<input_file> run
```

in the command line. To check the status of the job submission, issue

```bash
$ squeue -u dpassero
```

(since we're using Daniele's scratch).

Now, having generated the geometry file, we can optimize it. Use the following submission script, which is a modified version of previous scripts, except that we are now using one node partitioned into 48 cores:

```bash
#!/bin/bash

#SBATCH -N 1
#SBATCH -n 48
#SBATCH --time=4:00:00
#SBATCH --job-name="change me"
##SBATCH --mem-per-cpu=1024
#SBATCH --output=stdout.txt
#SBATCH --error=stderr.txt

module load gcc/6.3.0 openmpi/4.0.2 cp2k/8.2
mpirun cp2k.psmp -i benz_pt.inp > benz_pt.out
```

The name of the input file in the current directory is `benz_pt.inp`, to match with the last line of the submit script. Open it with `vim` to inspect it: notice how atoms `1..72` are kept fixed. This has practical (ie, computational) as well as physical reasons, such as saving computation time but also  Once done scrolling through it, close it again.

As we've done in previous exercises: extract this script, write it to a file (say, `submit.sh`), change the permissions, and submit the job:

```bash
$ chmod 755 submit.sh
$ sbatch submit.sh
```

#### Assignment 1: Optimization of the Pt slab
Perform the otimization of the slab: in order that we calculate the adsorption energy, the calculation must be repeated for the slab without the adsorbate:
1. Copy the input into a new one for the slab: `$ cp benzene_on_pt.inp pt_slab.inp`
2. Visualize the input file with `vim`: `$ vim pt_slab.inp`.
3. In the `GLOBAL` section, change the project name so that you do not overwrite the input files:
`PROJECT pt_slab`
4. In the `FORCE_EVAL` section, change the name of the `.xyz` file to `pt_slab.xyz` already in the present folder: `COORD_FILE_NAME pt_slab.xyz` and save. Next, in order to save time, we want to use the wavefunction generated from the previous calculation on the current calculation: `$ cp benzene_on_pt-RESTART.wfn pt_slab-RESTART.wfn`
5. Now we need to create the geometry file `pt_slab.xyz` by copying the geometry file: `$ cp benzene_on_pt.xyz pt_slab.xyz`
6. Open the `pt_slab.xyz` using `vim` and delete the coordinates of the adsorbate molecule, and change the number of atoms accordingly: benzene has 12 atoms, so there are a total of 32 atoms in the current slab. Open the file for use in ASE via the `read()` function, and visualize it using the pre-defined `view_structure()` function. **Very important:** Make sure to use the `get_cell()` method on this new slab (and indeed in all subsequent calculations) and change as appropriate the values of the `CELL` subsection in the CP2K input file!
7. Submit the calculation. The remaining parts of the input file should remain the same, because we want the simulations to be as similar as possible in order to compute the adsorption energy

**Important**: make sure to keep the same atoms fixed during the optimization of the slab!

In [ ]:
## -- your code here

#### End Assignment 1

#### Assignment 2: Optimization of the benzene molecule
Repeat the procedure outlined in Assignment 1 for the beneze molecule, changing
all names of files as necessary. **Important!** pay particular attention to the modification of the `.xyz` file: you might rename it something like `benzene.xyz`, so don’t
forget to change the number of atoms (first line) in the file, as well as to remove the
atoms you don’t want (in this case, the platinum atoms of the slab, since now we
just want the molecule).

In [ ]:
## -- Your code here

#### End Assignment 2

#### Assignment 3: Computation of the adsorption energy

Use your calculations from above and code from previous exercises to visualize the relaxation trajectory. We now have all the ingredients to compute the adsorption energy $E_{\text{ads}}$ of benzene on Pt(111) according to Eq. 1:

$$
E_{\text{ads}} = E_{\text{slab \& mol}} - E_{\text{slab}} - E_{\text{mol}}
$$

[Write a function to] compute the adsorption energy of benzene on the Pt(111) surface. The simplest and probably most straightforward way is to use the `grep` command to extract all instances of the total energy, then pipe this to the `tail` command (using the appropriate flags of course) to extract only the line containing last instance of the total energy (ie, the line converged value!), and finally to pipe this to the `awk '{print $3}'` command (the single ticks are crucial, do not forget them!), which will print the third column of this line, ie the total energy:

```bash
$ grep <expression for total energy> | tail <flags to extract only the last line> | awk '{print $3}'
```

In [ ]:
### Your code here

#### End Assignment 3

## 2. Methane $\text{CH}_4$ on Pt(111)

We now repeat the above procedure for methane on Pt(111):

In [100]:
methane=molecule('CH4')
meth_pt_slab = fcc111('Pt', size=(6, 6, 6))
add_adsorbate(meth_pt_slab, methane, height=2.25, position=(10, 7))
meth_pt_slab.center(vacuum=10, axis=2)
view_structure(meth_pt_slab)

NGLWidget()

In [101]:
meth_pt_slab.get_cell()

Cell([[16.6311514935076, 0.0, 0.0], [8.3155757467538, 14.402999687565085, 0.0], [0.0, 0.0, 34.195183276116666]])

In [102]:
write(filename='methane_on_pt_6layers.xyz',images=meth_pt_slab)

#### Assignment 4: The adsorption energy of methane on Pt(111)

Repeat Assignments 1-3 in order to arrive at the adsorption energy Eq.
1 for methane on Pt(111), whose full geometry file is given in the current directory as
`methane_on_pt.xyz`, which we have just generated. Hint: with all these different input and output files, it might
not be a bad idea to create separate directories to make everything a bit more
organised...

In [ ]:
## Your code here

#### End Assignment 4

#### Assignment 5: Adsorption of ethane
Repeat Assignments 1-3 in order to arrive at the adsorption energy Eq.
1 for ethane.

In [88]:
ethane=molecule('C2H6')
eth_pt_slab = fcc111('Pt', size=(6, 6, 3))
add_adsorbate(eth_pt_slab, ethane, height=1.75, position=(10, 7))
eth_pt_slab.center(vacuum=10, axis=2)
view_structure(eth_pt_slab)

NGLWidget()

Notice how the ethane molecule is "vertical" (the axis of the molecule is perpendicular to the surface). In the paper, the ethane molecule is horizontal. We have to rotate the molecule 90° about the $y$-axis:

In [89]:
ethane.rotate('y', 90)
view_structure(ethane)

NGLWidget()

In [90]:
eth_pt_slab = fcc111('Pt', size=(6, 6, 3))
add_adsorbate(eth_pt_slab, ethane, height=1.75, position=(10, 7))
eth_pt_slab.center(vacuum=10, axis=2)
view_structure(eth_pt_slab)

NGLWidget()

This looks more like the structure in the paper. Now we can save it:

In [91]:
write(filename='ethane_on_pt.xyz',images=eth_pt_slab)

/Users/ray/opt/miniconda3/envs/aiida/lib/python3.8/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


In [ ]:
## -- Your code here

#### End Assignment 5

#### Assignment 6: The BEEF-vdW exchange-correlation functional
We now want to investigate adsorption energies of each of the three above molecules
using the Bayesian error estimation functional with van der Waals correlation
(BEEF-vdW). We have met vdW forces in previous exercises, and hopefully it should
be interesting to investigate their roll in chemisorption.

Your task now is to repeat the above calculations, so that you obtain the adsorp-
tion energy Eq. 2 using now the BEEF-vdW exchange correlation functional. You
singly need to change the functional in the `&FORCE_EVAL` section from PBE to
BEEFVDW, according to: `&XC_FUNCTIONAL BEEFVDW`

Reproduce the bar graphs for benzene, methane, and ethane in Figs. 3 (a) and (b) of
the paper linked in the PDF. Since we did calculations only for the PBE and BEEF-vdW functionals,
of course these are the only ones you’ll plot. The experimental data for comparison
are given in Table S1 of the supplementary information of the paper (also linked in the PDF).

In [ ]:
## -- Your code here

#### End Assignment 6

#### OPTIONAL Assignment 7: identification of bonding pairs

In identifying chemi- resp. physisorption, one of the most important quantities are the distances (in the relaxed geometry!) between atoms of the adsorbate and those of the substrate. Pick any of the above molecules (benzene, methane, ethane) and use the `get_distances()` function to compare the distances of the closest 2-3 molecule-substrate atom pairs between the starting geometry and the final relaxed geometry. Comment on your results.

In [ ]:
## -- Your code here

#### End Assignment 7

#### OPTIONAL Assignment 8: computation of the deformation energy

**Notice**: completion of this assignment will result in a substantial number of extra points, for those who might have missed a few exercise submissions ;-)

In the above geometry optimizations, you've probably noticed that the molecule "buckles" a little bit: this is due to the molecule's interaction with the substrate, which in turn performs work on the molecule to reach a total energy minimum. The objective of this optional assignment will be to calculate mechanical (or really... electrostatic) work that the substrate performs on the molecule. This simply amounts to the energy difference between the relaxed molecule in the gasphase $E_{\text{gasphase}}^{\text{relaxed}}$ and the energy of *just the molecule* in the relaxed adsorption configuration on the surface. Since we already have $E_{\text{gasphase}}^{\text{relaxed}}$ from our calculations from previous steps, we just need to extract the energy of the molecule on the surface.

1. Choose the molecule for which you see the "most" deformation in the course of the geometry optimization on the platinum slab (e.g., benzene on platinum, methane on platinum, or ethane on platinum. In any case, make sure it's the *total* system geometry that you choose, and not just the slab nor just the molecule in this first step).
2. Take the relaxed geometry configuration (i.e., the last block of lines in the `-pos-1.xyz` file, including the number of atoms line and the comment line directly underneath) and store them to a new file: `tail -<however many lines> something-pos-1.xyz > final_config.xyz`.
3. Delete the atoms of the slab (so that only the atoms of the molecule remain), and adjust the total number of atoms (first line) accordingly.
3. Perform an `ENERGY` run on this `final_config.xyz` configuration: remember what an `ENERGY` run is --- all it does is simply compute the energy of a given configuration; it does *not* optimize the geometry. Perform an energy run by modifying the `RUN_TYPE` parameter in the input file from `GEO_OPT` to `ENERGY` and changing the name of the `COORD_FILE_NAME` parameter to `final_config.xyz`; it might also be sensible to change the name of the `PROJECT` in the `&GLOBAL` section. Once all these changes have been made, submit the calculation. Once it's finished, you have $E_{\text{mol on slab}}^{\text{relaxed}}$ 
4. Extract the difference in energy --- the **deformation energy** $E_{\text{defo}}$:
$$
E_{\text{defo}} = E_{\text{gasphase}}^{\text{relaxed}} - E_{\text{mol on slab}}^{\text{relaxed}}
$$

In [ ]:
## -- Your code here

#### End Assignment 8